In [66]:
from sklearn.datasets import fetch_openml

# Load Fashion MNIST
fashion_mnist = fetch_openml('Fashion-MNIST', version=1, as_frame=False)

# Extract data and labels
X, y = fashion_mnist.data, fashion_mnist.target.astype(int)

print(f"Data shape: {X.shape}, Labels shape: {y.shape}")


Data shape: (70000, 784), Labels shape: (70000,)


In [67]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn as nn
from sklearn.model_selection import train_test_split

In [68]:
torch.manual_seed(123)

In [69]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [70]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [71]:
y_train

array([8, 0, 6, ..., 7, 6, 6])

In [72]:
class dataset(Dataset):

  def __init__(self, x, y):

    self.x = torch.tensor(x, dtype=torch.float32)
    self.y = torch.tensor(y, dtype=torch.long)

  def __len__(self):
    return len(self.x)

  def __getitem__(self, index):
    return self.x[index], self.y[index]

In [73]:
train_dataset = dataset(x_train, y_train)
test_dataset = dataset(x_test, y_test)

In [82]:
train_loader = DataLoader(train_dataset, batch_size=128,pin_memory=True, shuffle=True) # use larger batch size and pin_memory
test_loader = DataLoader(test_dataset, batch_size=128,pin_memory=True, shuffle=True)

In [83]:
class Model(nn.Module):

  def __init__(self,num_features):
    super().__init__()

    self.layers = nn.Sequential(
        nn.Linear(num_features, 128),
        nn.ReLU(),
        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Linear(64, 10)
    )

  def forward(self, x):
    return self.layers(x)

In [84]:
model = Model(x_train.shape[1]).to(device)

In [85]:
lr = 0.001
epochs = 25

In [86]:
lossfn = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=lr)

In [87]:
device

device(type='cuda')

In [88]:
for i in range(epochs):

  total_epoch_loss = 0

  for batch_features, batch_labels in train_loader:

    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    output = model(batch_features)

    loss = lossfn(output, batch_labels)

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()

    total_epoch_loss += loss.item()

  print(f"Epoch: {i+1}, Loss: {total_epoch_loss/len(train_loader)}")


Epoch: 1, Loss: 0.8273949398569864
Epoch: 2, Loss: 0.43790203938473304
Epoch: 3, Loss: 0.39738929122005967
Epoch: 4, Loss: 0.3774742960045327
Epoch: 5, Loss: 0.34840084731442744
Epoch: 6, Loss: 0.3439651353106107
Epoch: 7, Loss: 0.32977073537568524
Epoch: 8, Loss: 0.3251156220085
Epoch: 9, Loss: 0.31295329427610247
Epoch: 10, Loss: 0.31714767037325253
Epoch: 11, Loss: 0.31025493985305636
Epoch: 12, Loss: 0.29447864361753745
Epoch: 13, Loss: 0.30170230193225217
Epoch: 14, Loss: 0.29325156116254253
Epoch: 15, Loss: 0.2941481983416701
Epoch: 16, Loss: 0.2862060273823128
Epoch: 17, Loss: 0.2809003053326585
Epoch: 18, Loss: 0.2760756750969582
Epoch: 19, Loss: 0.2754505916273213
Epoch: 20, Loss: 0.27218134388259557
Epoch: 21, Loss: 0.2663730951919131
Epoch: 22, Loss: 0.26379324162387413
Epoch: 23, Loss: 0.2613833422121936
Epoch: 24, Loss: 0.2640929298824099
Epoch: 25, Loss: 0.2562598446256494


In [89]:
model.eval()

Model(
  (layers): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [91]:
total = 0
count = 0

with torch.no_grad():

  for batch_features, batch_labels in test_loader:

    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    output = model(batch_features)

    _, predicted = torch.max(output.data, 1)

    total += batch_labels.size(0)

    count += (predicted == batch_labels).sum().item()

print(f"Accuracy: {count/total}")

Accuracy: 0.8774285714285714
